# AI Research Multi-Agent System

This notebook demonstrates a multi-agent AI research system using Microsoft Agent Framework (Autogen).

## Architecture
- **GPT-5.2 Search Agent**: Primary research agent with Bing search grounding
- **Grok 4 Search Agent**: Secondary research agent with Bing search grounding
- **Claude Mediator**: Arbiter/critique model that cross-questions and validates results

## Setup
1. Copy `.env.example` to `.env` and fill in your API keys
2. Install dependencies: `pip install -r requirements.txt`
3. Run this notebook

In [ ]:
# Import required libraries
import os
import json
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager

# Load environment variables
load_dotenv()

print("Environment loaded successfully!")

## Configuration

Set up the LLM configurations for each agent.

In [ ]:
# GPT-5.2 Configuration (Azure OpenAI)
gpt52_config = {
    "config_list": [
        {
            "model": os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-52"),
            "api_type": "azure",
            "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
            "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
            "api_version": os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01"),
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
}

# Grok 4 Configuration (xAI)
grok4_config = {
    "config_list": [
        {
            "model": os.getenv("XAI_GROK_MODEL", "grok-4"),
            "api_key": os.getenv("XAI_API_KEY"),
            "base_url": "https://api.x.ai/v1",
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
}

# Claude Configuration (Anthropic)
claude_config = {
    "config_list": [
        {
            "model": os.getenv("ANTHROPIC_CLAUDE_MODEL", "claude-3-opus-20240229"),
            "api_key": os.getenv("ANTHROPIC_API_KEY"),
            "api_type": "anthropic",
        }
    ],
    "temperature": 0.5,
    "timeout": 120,
}

print("Configurations loaded!")

## Bing Search Tool

Create a Bing search function for grounding the search agents.

In [ ]:
import requests

def bing_search(query: str, count: int = 5) -> str:
    """
    Perform a Bing web search and return results.
    
    Args:
        query: Search query string
        count: Number of results to return (default: 5)
    
    Returns:
        Formatted string with search results
    """
    api_key = os.getenv("BING_SEARCH_API_KEY")
    endpoint = os.getenv("BING_SEARCH_ENDPOINT", "https://api.bing.microsoft.com/v7.0/search")
    
    if not api_key:
        return "Error: BING_SEARCH_API_KEY not configured"
    
    headers = {"Ocp-Apim-Subscription-Key": api_key}
    params = {"q": query, "count": count, "mkt": "en-US"}
    
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        search_results = response.json()
        
        # Format results
        results = []
        if "webPages" in search_results and "value" in search_results["webPages"]:
            for idx, result in enumerate(search_results["webPages"]["value"], 1):
                results.append(
                    f"{idx}. {result['name']}\n"
                    f"   URL: {result['url']}\n"
                    f"   Snippet: {result.get('snippet', 'N/A')}\n"
                )
        
        return "\n".join(results) if results else "No results found."
    
    except Exception as e:
        return f"Error performing search: {str(e)}"

# Register the function for use by agents
print("Bing search function defined!")

## Create Research Agents

Define the multi-agent system with specialized roles.

In [ ]:
# GPT-5.2 Search Agent
gpt_researcher = ConversableAgent(
    name="GPT_Researcher",
    system_message="""You are a primary research agent powered by GPT-5.2.
    Your role is to conduct thorough research on given topics using web search.
    When researching:
    1. Use the bing_search function to find relevant information
    2. Analyze and synthesize information from multiple sources
    3. Provide well-structured, evidence-based insights
    4. Cite your sources clearly
    5. Be open to critique and validation from other agents
    """,
    llm_config=gpt52_config,
    human_input_mode="NEVER",
)

# Grok 4 Search Agent
grok_researcher = ConversableAgent(
    name="Grok_Researcher",
    system_message="""You are a secondary research agent powered by Grok 4.
    Your role is to provide alternative perspectives and complementary research.
    When researching:
    1. Use the bing_search function to find relevant information
    2. Focus on different angles than the primary researcher
    3. Look for counterarguments and alternative viewpoints
    4. Validate or challenge findings from other agents
    5. Provide unique insights and analysis
    """,
    llm_config=grok4_config,
    human_input_mode="NEVER",
)

# Claude Mediator/Arbiter Agent
claude_mediator = ConversableAgent(
    name="Claude_Mediator",
    system_message="""You are a mediator and arbiter powered by Claude.
    Your role is to:
    1. Cross-question the search agents to probe their findings
    2. Identify inconsistencies or gaps in their research
    3. Validate results by comparing different agents' outputs
    4. Synthesize a final, well-reasoned answer
    5. Ask clarifying questions when needed
    6. Ensure the research is comprehensive and balanced
    
    You should be critical, thorough, and impartial in your mediation.
    """,
    llm_config=claude_config,
    human_input_mode="NEVER",
)

# User proxy for initiating research
user_proxy = ConversableAgent(
    name="User",
    system_message="You are a user who initiates research tasks.",
    llm_config=False,  # No LLM needed for user proxy
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", ""),
)

# Register the bing_search function with all agents
for agent in [gpt_researcher, grok_researcher, claude_mediator]:
    agent.register_for_llm(
        name="bing_search",
        description="Search the web using Bing to find relevant information on a topic."
    )(bing_search)

for agent in [user_proxy]:
    agent.register_for_execution(name="bing_search")(bing_search)

print("Agents created successfully!")

## Set up Group Chat

Create a group chat where agents can collaborate.

In [ ]:
# Create group chat
group_chat = GroupChat(
    agents=[user_proxy, gpt_researcher, grok_researcher, claude_mediator],
    messages=[],
    max_round=12,
    speaker_selection_method="auto",
)

# Create group chat manager
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=gpt52_config,  # Use GPT-5.2 for managing the conversation flow
)

print("Group chat configured!")

## Research Function

Main function to conduct research on a given topic.

In [ ]:
def conduct_research(topic: str, max_rounds: int = 12) -> Dict[str, Any]:
    """
    Conduct multi-agent research on a given topic.
    
    Args:
        topic: Research topic/question
        max_rounds: Maximum number of conversation rounds
    
    Returns:
        Dictionary containing research results and conversation history
    """
    # Update group chat max rounds
    group_chat.max_round = max_rounds
    
    # Initiate the research
    research_prompt = f"""
    Research Topic: {topic}
    
    Instructions for the team:
    1. GPT_Researcher: Begin by searching for and analyzing information on this topic
    2. Grok_Researcher: Provide complementary research and alternative perspectives
    3. Claude_Mediator: Cross-question both researchers, validate findings, and synthesize final answer
    
    Please collaborate to provide a comprehensive, well-researched answer.
    End with "TERMINATE" when the research is complete.
    """
    
    # Start the conversation
    chat_result = user_proxy.initiate_chat(
        manager,
        message=research_prompt,
    )
    
    return {
        "topic": topic,
        "chat_history": chat_result.chat_history,
        "summary": chat_result.summary,
    }

print("Research function ready!")

## Example: Conduct Research

Let's research a topic using our multi-agent system.

In [ ]:
# Example research topic
research_topic = "What are the latest advancements in quantum computing in 2024?"

print(f"Starting research on: {research_topic}")
print("=" * 80)

# Conduct the research
results = conduct_research(research_topic, max_rounds=15)

print("\n" + "=" * 80)
print("Research Complete!")
print("=" * 80)

## View Research Results

Display the conversation and final summary.

In [ ]:
# Display chat history
print("\nConversation History:")
print("=" * 80)
for i, message in enumerate(results["chat_history"], 1):
    speaker = message.get("name", "Unknown")
    content = message.get("content", "")
    print(f"\n[{i}] {speaker}:")
    print("-" * 80)
    print(content)
    print()

# Display summary
print("\n" + "=" * 80)
print("Summary:")
print("=" * 80)
print(results.get("summary", "No summary available"))

## Custom Research Query

Use this cell to research your own topic.

In [ ]:
# Enter your research question here
my_topic = "Your research question here"

# Uncomment to run
# my_results = conduct_research(my_topic, max_rounds=15)
# print("Research complete! Check the results above.")

## Export Results

Save research results to a file.

In [ ]:
import json
from datetime import datetime

def save_research_results(results: Dict[str, Any], filename: Optional[str] = None):
    """
    Save research results to a JSON file.
    
    Args:
        results: Research results dictionary
        filename: Output filename (defaults to timestamped file)
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"research_results_{timestamp}.json"
    
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"Results saved to: {filename}")

# Example: Save the results
# save_research_results(results)
print("Export function ready!")

## Advanced Configuration

Customize agent behavior and conversation flow.

In [ ]:
# You can modify agent configurations here
# For example, adjust temperature, add custom system messages, etc.

def create_custom_researcher(name: str, system_message: str, config: Dict[str, Any]):
    """
    Create a custom research agent with specific configuration.
    
    Args:
        name: Agent name
        system_message: Custom system message
        config: LLM configuration
    
    Returns:
        ConversableAgent instance
    """
    agent = ConversableAgent(
        name=name,
        system_message=system_message,
        llm_config=config,
        human_input_mode="NEVER",
    )
    
    # Register search function
    agent.register_for_llm(
        name="bing_search",
        description="Search the web using Bing to find relevant information."
    )(bing_search)
    
    return agent

print("Custom agent creation function ready!")

## Troubleshooting

Common issues and solutions:

1. **API Key Errors**: Ensure all API keys are properly set in your `.env` file
2. **Search Not Working**: Verify Bing Search API key and endpoint are correct
3. **Timeout Errors**: Increase the `timeout` value in agent configurations
4. **Rate Limits**: Add delays between requests if hitting API rate limits

For more help, refer to:
- [Microsoft AutoGen Documentation](https://microsoft.github.io/autogen/)
- [Azure AI Documentation](https://learn.microsoft.com/azure/ai-services/)
- [Anthropic Claude API](https://docs.anthropic.com/)
- [xAI Documentation](https://docs.x.ai/)